In [2]:
import numpy as np
import numpy.linalg as LA
from scipy.spatial.transform import Rotation as R
from rich.progress import track
import matplotlib.pyplot as plt

In [6]:
class sgeom_coordinate:
    def __init__(
        self,
        clen,
        poni,
        pixel_size,
        X1=[0, 0, 1],
        X2=[1, 0, 0],
        X3=[0, 1, 0],
        rot=[0.0, 0.0, 0.0],
    ):
        self.clen = clen
        self.poni = poni + [0]
        self.pixel_size = pixel_size

        # Lab direction vectors by sample vectors
        self.X1_hat = np.array(X1) / LA.norm(X1)  # Sample z
        self.X2_hat = np.array(X2) / LA.norm(X2)  # Sample x
        self.X3_hat = np.array(X3) / LA.norm(X3)  # Sample y
        # Det coordinate expressed by lab vectors
        d1_hat_lab = np.array([1, 0, 0])
        d2_hat_lab = np.array([0, 1, 0])
        d3_hat_lab = np.array([0, 0, 1])
        r = R.from_rotvec([[-rot[0], 0, 0], [0, -rot[1], 0], [0, 0, rot[2]]])
        for i, rotation in enumerate(r):
            d1_hat_lab = rotation.apply(d1_hat_lab)
            d2_hat_lab = rotation.apply(d2_hat_lab)
            d3_hat_lab = rotation.apply(d3_hat_lab)
        self.d1_hat_lab = d1_hat_lab
        self.d2_hat_lab = d2_hat_lab
        self.d3_hat_lab = d3_hat_lab

    def detector_to_lab_coords(
        self, det_pixels: list[list[int]]
    ) -> list[np.ndarray[float]]:
        converted = []
        for pixel in det_pixels:
            pixel_info = np.append(pixel, 0)
            without_rotate = np.array([self.pixel_size, self.pixel_size, 0]) * (
                pixel_info
            ) + np.array([-self.poni[0], -self.poni[1], self.clen])

            converted.append(
                without_rotate[0] * self.d1_hat_lab
                + without_rotate[1] * self.d2_hat_lab
                + without_rotate[2] * self.d3_hat_lab
            )
        return converted

    def lab_to_sample_coords(
        self, lab_coords: list[list[int | float]]
    ) -> list[np.ndarray[float]]:
        return [
            coord[0] * self.X1_hat + coord[1] * self.X2_hat + coord[2] * self.X3_hat
            for coord in lab_coords
        ]

    def detector_to_sample_coords(self, det_pixels: list[list[int]]):
        return self.lab_to_sample_coords(self.detector_to_lab_coords(det_pixels))

In [ ]:
coord = sgeom_coordinate(
    X1=[0, 0, 1],
    X2=[1, 0, 0],
    X3=[0, 1, 0],
    rot=np.pi * np.array([0.5, 0, 0]),
    clen=0.13,
    poni=[0.12, 0.12],
    pixel_size=2e-4,
)
print((lab := coord.detector_to_lab_coords(det_pixels=[[0, 0], [1, 1]])))
print(coord.lab_to_sample_coords(lab))

In [ ]:
def get_q_values(
    pixels: list[list[int]],
    geom: sgeom_coordinate,
    beam_center: list[float],
    wavelength: float = 12.398 / 9.3,
):
    ewald_radius = 2 * np.pi / wavelength
    lab_coords = geom.detector_to_lab_coords(pixels)

    beam_center = geom.detector_to_lab_coords([np.divide(beam_center, geom.pixel_size)])
    assert len(beam_center) == 1
    beam_center = beam_center[0]

    k0 = beam_center / LA.norm(beam_center) * ewald_radius

    q_values = []
    for coord in lab_coords:
        q_values.append(coord / LA.norm(coord) * ewald_radius - k0)
    return q_values

In [ ]:
def generate_pixel_values(step=5):
    x = []
    for i in track(range(0, 1200, step)):
        x.append(
            get_q_values([[i, o] for o in range(0, 1200, step)], coord, [0.12, 0.12])
        )
    return np.array(x)

In [ ]:
data = generate_pixel_values(step=5)
data *= 1 / data.max()
normed = np.apply_along_axis(LA.norm, 2, data)
g = plt.imshow(normed, origin="lower")